**Author:** Anowar Shajib

In [ ]:
try:
  import ipywidgets
except:
  !pip install ipywidgets

  import ipywidgets

In [ ]:
try:
  import lenstronomy
except:
  !pip install lenstronomy

  import lenstronomy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import copy


from ipywidgets import interactive

from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.ImSim.image2source_mapping import Image2SourceMapping
from lenstronomy.Util.param_util import phi_q2_ellipticity

## function definitions

In [ ]:
def make_lens_image(
    theta_E=1.,
    gamma=2.,
    phi_lens=180, 
    q_lens=0.5,
    center_x_lens=0.,
    center_y_lens=0.,
    x_source=0.2, y_source=0.2, 
    amp_source=10,
    phi_source=180, q_source=0.2,
    R_sersic_source=0.2,
    lens_galaxy=True):
    """
    """
            
    lens_model_list = ['EPL']
    
    if lens_galaxy:
        mult = 1
    else:
        mult = 1e-6
    
    e1_lens, e2_lens = phi_q2_ellipticity(phi_lens*np.pi/180, q_lens)
            
    kwargs_lens = [{'theta_E': theta_E * mult,
                    'gamma': gamma, 
                    'e1': e1_lens,
                    'e2': e2_lens, 
                    'center_x': center_x_lens,
                    'center_y': center_y_lens}]

    lens_model = LensModel(lens_model_list=lens_model_list)
    
    lens_light_model = LightModel(['SERSIC_ELLIPSE'])

    kwargs_lens_light = [{'amp': 16, 
                          'R_sersic': 0.6 * theta_E, 
                          'n_sersic': 4.0, 
                          'e1': e1_lens, 
                          'e2': e2_lens, 
                          'center_x': center_x_lens, 
                          'center_y': center_y_lens}]
    
    
    source_model_list = ['SERSIC_ELLIPSE']
    e1, e2 = phi_q2_ellipticity(phi_source*np.pi/180, q_source)
    kwargs_sersic_ellipse =  {'amp': amp_source,
                              'R_sersic': R_sersic_source,
                              'n_sersic': 1.0,
                              'e1': e1, 'e2': e2,
                              'center_x': x_source,
                              'center_y': y_source}


    kwargs_source = [kwargs_sersic_ellipse]
    source_model = LightModel(light_model_list=source_model_list)
    
    
    w = 5
    N = 100
    x, y = np.meshgrid(np.linspace(-w, w, N), np.linspace(-w, w, N))
    
    lens_light = lens_light_model.surface_brightness(x, y, kwargs_lens_light).reshape((N, N))
    
    image_mapper = Image2SourceMapping(lens_model, source_model)
    image = image_mapper.image_flux_joint(x.flatten(), y.flatten(), kwargs_lens,
                              kwargs_source).reshape((N, N))
    image[image < 0] = 1e-6
    
    if lens_galaxy:
        image += lens_light
        
    return image


def plot_lens_simulation(
    theta_E=1.,
    gamma=2.,
    phi_lens=180, 
    q_lens=0.5,
    center_x_lens=0.,
    center_y_lens=0.,
    x_source=0.2, y_source=0.2, 
    amp_source=10,
    phi_source=180, q_source=0.2,
    R_sersic_source=0.2,
    lens_galaxy=True):
    """
    """
    image = make_lens_image(
        theta_E,
        gamma,
        phi_lens, 
        q_lens,
        center_x_lens,
        center_y_lens,
        x_source, y_source, 
        amp_source,
        phi_source, q_source,
        R_sersic_source, lens_galaxy
    )
    
    fig, axes = plt.subplots(1, 1, figsize=(15,5))

    axes.matshow(np.log10(image), origin='lower', cmap='cubehelix', vmin=0, vmax=3)
    
    plt.show()

## Simulating a lens system

Move around the sliders to see how the lensing system changes depending on the lens galaxy and source galaxy's properties.


Here're a few definitions for the parameters used. The form of the surface mass density profile is given by:
$$\Sigma(x, y) \propto \frac{3 - \gamma}{2} \left[ \frac{\theta_{\rm E}}{\sqrt{q
(x-x_0)^2 + (y-y_0)^2 / q}} \right]^{\gamma - 1}. $$

The position angle $\phi$ adjusts the orientation of the mass profile's
major axis.

The source galaxy's light profiles are modeled
with Sersic function, which is given by:

$$I(x, y) = I_{\rm e} \exp \left[ -b_{n} \left\{ \left( \frac{\sqrt{q(x-x_{\rm s})^2
+ (y - y_{\rm s})^2/q}}{R_{\rm Sersic}} \right)^{1/n_{\rm Sersic}} - 1 \right\}
\right].$$

Here, $I_{\rm e}$ is the amplitude and $n_{\rm Sersic}$ is fixed to 1. The lens galaxy's light profile is also a Sersic function, but that is automatically adjusted with the $\theta_{\rm E}$ value, so you don't need to tune the lens galaxy's light profile.

In [ ]:
interactive_plot = interactive(plot_lens_simulation, 
                               theta_E=(0.02, 3., 0.02),
                               gamma=(1.6, 2.4, 0.01),
                               phi_lens=(0, 90, 1), 
                               q_lens=(0.2, 1., 0.01),
                               center_x_lens=(-5, 5, 0.1),
                               center_y_lens=(-5, 5, 0.1),
                               x_source=(-5, 5, 0.1), 
                               y_source=(-5, 5, 0.1), 
                               amp_source=(5, 1000, 5),
                               phi_source=(0, 90, 1), 
                               q_source=(0.2, 1., 0.01),
                               R_sersic_source=(0.01, 2, 0.01),
                               lens_galaxy=True
                              )

output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

## Demonstration of lens modeling by tuning parameters by hand

Here we will work with imaging data of lensing systems given here: https://drive.google.com/drive/folders/18sPD9tpPpTHrNBrDbN6oIX3a3F1tTcZf?usp=share_link. 

Load the data corresponding to your team number and order.


In [ ]:
# provide address of your input image file here
file_name = '../data/mock_lens_images/team_1_order_1.txt'

In [ ]:
input_image = np.loadtxt(file_name)

temp_copy = copy.deepcopy(input_image)
temp_copy[temp_copy < 0] = 1e-6
noise = np.sqrt(temp_copy/300)
noise = np.sqrt(noise**2 + 1.**2)


def plot_model_fitting(
    theta_E=1.,
    gamma=2.,
    phi_lens=180, 
    q_lens=0.5,
    center_x_lens=0.,
    center_y_lens=0.,
    x_source=0.2, y_source=0.2, 
    amp_source=10,
    phi_source=180, q_source=0.2,
    R_sersic_source=0.2,
    lens_galaxy=True):
    """
    """
    image = make_lens_image(
        theta_E,
        gamma,
        phi_lens, 
        q_lens,
        center_x_lens,
        center_y_lens,
        x_source, y_source, 
        amp_source,
        phi_source, q_source,
        R_sersic_source,
        lens_galaxy
    )
    
    fig, axes = plt.subplots(1, 3, figsize=(15,5))

    axes[0].matshow(np.log10(input_image), origin='lower', cmap='cubehelix', vmin=0)
    axes[0].set_title('Data')
    axes[0].set_aspect('equal')

    axes[1].matshow(np.log10(image), origin='lower', cmap='cubehelix', vmin=0)
    axes[1].set_title('Model')
    #     axes[1].set_aspect('equal')

    axes[2].matshow((input_image - image)/noise, vmin=-3, vmax=3, cmap='RdBu_r', origin='lower')
    axes[2].set_title('Normalized residual, red. chi^2={:.2f}'.format(np.mean(((input_image - image)/noise)**2)))
    #     axes[2].set_aspect('equal')
        
    plt.show()

In [ ]:
interactive_plot = interactive(plot_model_fitting, 
                               theta_E=(0.02, 3., 0.02),
                               gamma=(1.6, 2.4, 0.01),
                               phi_lens=(0, 90, 1), 
                               q_lens=(0.2, 1., 0.01),
                               center_x_lens=(-5, 5, 0.1),
                               center_y_lens=(-5, 5, 0.1),
                               x_source=(-5, 5, 0.1), 
                               y_source=(-5, 5, 0.1), 
                               amp_source=(5, 1000, 5),
                               phi_source=(0, 90, 1), 
                               q_source=(0.2, 1., 0.01),
                               R_sersic_source=(0.01, 2, 0.01),
                               lens_galaxy=True
                              )

output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot
